In [0]:
import pandas as pd
import numpy as np
import glob
import os
import scipy.stats as ss
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from __future__ import print_function
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics.cluster import homogeneity_score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.cm as cm
%matplotlib inline

In [0]:
# ! pip install fancyimpute

In [0]:
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler, IterativeImputer

In [9]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Read data and impute missing values
#os.chdir('/content/drive/My Drive/Colab Notebooks/PhysioNet_Sepsis_Challenge')
# os.chdir('/content/drive/My Drive/Colab Notebooks/Physionet_sepsis_project/Data/Data_partial')
# os.chdir('/content/drive/My Drive/Colab Notebooks/Physionet_sepsis_project/Data/Data_full')
os.chdir('/content/drive/My Drive/Colab Notebooks/Physionet_sepsis_project/Data/Data_full')
# /content/drive/My Drive/Colab Notebooks/Demo_Physionet/Data

i = 0
df = pd.DataFrame()
for file in glob.iglob('*.psv'):
    f_name = float(os.path.splitext(os.path.basename(file))[0][1:])  
    # print(f_name)  
    tempdf = pd.read_csv(file, sep = '|', index_col = None, header = 0)    
    tempdf['Hour'] = tempdf.index
    tempdf['Identifier'] = f_name
    df = pd.concat([df, tempdf], axis=0)
    i = i+1

# Names of all columns in the data that contain physiological data
physiological_cols = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2',
       'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN',
       'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
       'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium',
       'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC',
       'Fibrinogen', 'Platelets']

# Names of all columns in the data that contain demographic data
demographic_cols = ['Age', 'Gender', 'Unit1', 'Unit2', 'HospAdmTime', 'ICULOS']

# Columns of features #730
feature_cols = physiological_cols + ['Hour'] + demographic_cols + ['Identifier']

# The name of the column that contains the value we are trying to predict
label_col = 'SepsisLabel'

#cols = list(df)
cols = feature_cols + [label_col]

# Move the SepsisLabel column to end of dataframe
#cols.insert(len(cols), cols.pop(cols.index('SepsisLabel')))
df = df.loc[:,cols]
# Plot percentage of missing values (NaNs) for each feature
cutoff = 60
fig = plt.figure(figsize=(20,10))
percent_missing = (df.isna().sum()/df.shape[0])*100
percent_missing.plot(kind="bar")
plt.plot(percent_missing, np.array([cutoff for i in range(len(percent_missing))]), 'r--') 
fig.suptitle('Percentage Missing Values', fontsize=20)
plt.xlabel('Feature', fontsize=16)
plt.ylabel('% Missing Values', fontsize=16)
plt.savefig('histogram.jpg')

# Retain columns in dataframe with <= cutoff% missing values 
df = df.loc[:, df.columns[percent_missing <= cutoff]]
print('Retained features:')
print(df.columns.values)

feature_cols = df.columns.values[:-1]
# Adjust physiological and demographic column names

physiological_cols = [x for x in feature_cols if x in set(physiological_cols)]
demographic_cols = [x for x in feature_cols if x in set(demographic_cols)]

# Impute missing data using fancyimpute package 
df_filled = pd.DataFrame(data = IterativeImputer().fit_transform(df[df.columns[:-1]].to_numpy()),
                         columns = df.columns[:-1],
                         index = df.index)


pd.set_option('display.expand_frame_repr', False)
df_filled['SepsisLabel'] = df['SepsisLabel']
print(df_filled.head())
print("files scanned",i)


In [0]:
continuous_cols = list(df_filled[df_filled.columns[0:6]].columns.values)
ordinal_cols = list(['Hour', 'Age','HospAdmTime','ICULOS'])
# binary_cols = list(['Gender', 'Unit1', 'Unit2'])
binary_cols = list(['Gender'])
# print(df_filled["Hour"].head())
df_filled[ordinal_cols] = df_filled[ordinal_cols].round()
df_filled[binary_cols] = df_filled[binary_cols].round()

sepsis = df_filled["SepsisLabel"]

In [0]:
# Calculate correlation between continuous & continuous variables
# using Pearson's Rho
print(continuous_cols)
print(ordinal_cols) 
corr = pd.DataFrame(columns = continuous_cols + ordinal_cols + binary_cols,
                    index =  continuous_cols + ordinal_cols + binary_cols)
con_con_corr = df_filled[continuous_cols].corr(method ='pearson')
corr[continuous_cols] = con_con_corr
print(corr.columns)

In [0]:
# Function to calculate correlation between (continuous & ordinal) and 
# (continuous & binary) variables using Correlation Ratio

def correlation_ratio(categories, measurements):
    fcat, _ = pd.factorize(categories)
    cat_num = np.max(fcat)+1
    y_avg_array = np.zeros(cat_num)
    n_array = np.zeros(cat_num)
    for i in range(0,cat_num):
        cat_measures = measurements[np.argwhere(fcat == i).flatten()]
        n_array[i] = len(cat_measures)
        y_avg_array[i] = np.average(cat_measures)
    y_total_avg = np.sum(np.multiply(y_avg_array,n_array))/np.sum(n_array)
    numerator = np.sum(np.multiply(n_array,np.power(np.subtract(y_avg_array,y_total_avg),2)))
    denominator = np.sum(np.power(np.subtract(measurements,y_total_avg),2))
    if numerator == 0:
        eta = 0.0
    else:
        eta = np.sqrt(numerator/denominator)
    return eta

In [0]:
# Calculate correlation between (continuous & ordinal) and 
# (continuous & binary) variables using Correlation Ratio
for con_col_name in continuous_cols:

  for bin_col_name in binary_cols:
    val = correlation_ratio(df_filled[con_col_name].values,
                            df_filled[bin_col_name].values)
    
    corr.loc[con_col_name, bin_col_name] = val
    corr.loc[bin_col_name, con_col_name] = val
  for ord_col_name in ordinal_cols:
    val = correlation_ratio(df_filled[con_col_name].values,
                            df_filled[ord_col_name].values)
    
    corr.loc[con_col_name, ord_col_name] = val
    corr.loc[ord_col_name, con_col_name] = val
print(corr)


In [0]:
# Function to calculate correlation between (ordinal & ordinal),
# (ordinal, binary) and (binary & binary) variables using
# Cramer's V 

def cramers_phi(x, y):
    confusion_matrix = pd.crosstab(x,y)
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
    rcorr = r-((r-1)**2)/(n-1)
    kcorr = k-((k-1)**2)/(n-1)
    return np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))

# Plot the correlation matrix
#sns.heatmap(corr, annot = True) 

In [0]:
# Calculate correlation between (ordinal & ordinal) and
# (binary & binary) variables using Cramer's V
for col in [ordinal_cols, binary_cols]:
  for i in range(0,len(col)):
    for j in range(i,len(col)):
      if i == j:                 
        corr.loc[col[i],col[j]] = 1.0            
      else:                 
        val = cramers_phi(df_filled[col[i]].values,
                          df_filled[col[j]].values) 
        corr.loc[col[i], col[j]] = val
        corr.loc[col[j], col[i]] = val

# Calculate correlation between (ordinal & binary)
# variables using Cramer's V
iteration = 0
for ord_col_name in ordinal_cols:
  for bin_col_name in binary_cols:
    val = cramers_phi(df_filled[ord_col_name].values,
                      df_filled[bin_col_name].values) 
    corr.loc[ord_col_name, bin_col_name] = val
    corr.loc[bin_col_name, ord_col_name] = val
    iteration = iteration+1

print(corr)
print("iterations",iteration)

In [0]:

# generate the linkage matrix
Z = linkage(corr, 'ward')

In [0]:
# Calculate full dendrogram 
plt.figure(figsize=(12, 12))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Variables')
plt.ylabel('distance')
dendrogram(Z, labels=list(corr.columns))
plt.axhline(y=2, color='r', linestyle='--')
plt.show()
plt.savefig('dendogram.jpg')

In [0]:
# Calculate cluster scores using PCA
pca = PCA(n_components = 1)
# Do the following for each cluster to get the cluster scores which will be
# the new features. 
# cluster_data = df_filled["#give the column names corresponding to the cluster#"].values
# cluster_data_1 = df_filled[["Hour","ICULOS"]].values
# cluster_data_2 = df_filled[["DBP","SBP","MAP"]].values
# cluster_data_3 = df_filled[["HR","Resp","O2Sat","HospAdmTime","Age","Gender"]].values
cluster_data_1 = df_filled[["Hour","ICULOS"]].values
cluster_data_2 = df_filled[["HospAdmTime","DBP","SBP","MAP"]].values
cluster_data_3 = df_filled[["HR","Resp","O2Sat","Age","Gender"]].values
cluster_score_1 = pca.fit_transform(StandardScaler().fit_transform(cluster_data_1))                  
cluster_score_2 = pca.fit_transform(StandardScaler().fit_transform(cluster_data_2))                  
cluster_score_3 = pca.fit_transform(StandardScaler().fit_transform(cluster_data_3))                  
# (cluster_score_1+cluster_score_2+cluster_score_3)
x = np.array([cluster_score_1.flatten(),cluster_score_2.flatten(),cluster_score_3.flatten()])
X = x.T
y = sepsis.to_numpy()
print(X.shape)
print(y.shape)
print(type(y))

In [0]:
from sklearn.model_selection import GroupKFold, StratifiedKFold, train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.tree import DecisionTreeClassifier
group = df_filled["Identifier"].to_numpy()
train_pred = []
train_actual = []

test_pred = []
test_actual = []

kf = GroupKFold(n_splits=5)
for train_idx, test_idx in kf.split(X, y, group):
   X_train, y_train = X[train_idx, :], y[train_idx]
   X_test, y_test = X[test_idx, :], y[test_idx]
   
   # Decision tree classifier with higher penalty
   # for misclassifying the low frequency output
   # label
   clf = DecisionTreeClassifier(class_weight="balanced",
                                max_depth=20,
                                max_leaf_nodes=20)
   model = clf.fit(X_train, y_train)

   train_pred.extend(clf.predict(X_train))
   train_actual.extend(y_train)

   test_pred.extend(clf.predict(X_test))
   test_actual.extend(y_test)



# Function for evaluating train and test accuracy
def evaluate(actual, predicted, prefix=""):
    precision = precision_score(actual, predicted)
    recall = recall_score(actual, predicted)
    accuracy = accuracy_score(actual, predicted)

    print("%s Precision: %.3f%%, Recall: %.3f%%, Accuracy: %.3f%%" % (prefix, precision * 100, recall * 100, accuracy * 100))



# Evaluate train and test accuracy
evaluate(train_actual, train_pred, "Train")
evaluate(test_actual, test_pred, "Test")

